In [0]:
import pandas as pd
import numpy as np

In [0]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678568 sha256=0c04efd30a85bd953aa4be3547fbcc91cf5337f17054a80b8936db8a6aae1241
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import surprise
from surprise import SVD, SVDpp, SlopeOne, KNNWithMeans
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from surprise import accuracy
from surprise.accuracy import rmse

In [0]:
import pickle

In [0]:
from sklearn import model_selection

In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir('/content/gdrive/My Drive/Analytics in Financial Services Project')

In [0]:
os.getcwd()

'/content/gdrive/My Drive/Analytics in Financial Services Project'

# Reading Datasets

In [0]:
transactions = pd.read_csv('Transaction_Final.csv')

In [0]:
demography_high = pd.read_csv('High_RFMSegment_Final.csv')

In [0]:
txn_demo_high = pd.read_csv('HighRFMSegment_Transactions_Join.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
txn_demo_high.columns[17]

'CreditLimitBand'

In [0]:
txn_demo_high['CreditLimitBand'].unique()

array(['01 (< 10)K', '02 (10 - 20)K', '03 (20 - 30)K', '05 (40 - 50)K',
       '04 (30 - 40)K', '11 (10 - 150)K', '07 (60 - 70)K',
       '10 (90 - 100)K', '14 (unknown)', '09 (80 - 90)K', '06 (50 - 60)K',
       '08 (70 - 80)K', '13 (>= 200)K', '12 (150 - 200)K'], dtype=object)

In [0]:
transactions.shape, demography_high.shape,txn_demo_high.shape

((374035, 13), (6414, 21), (242853, 33))

In [0]:
transactions.head()

,PartyID,CardType,Merchant_Category,Merchant_Name_Cleaned,Transaction_amount,Transaction_Date,HasTransacted?,WeekOfYear,Month,DayOfMonth,DayOfWeek,Weekday/Weekend,Merchant_Gateway
0,174884,Co-Brand,RESTAURANTS-BARS,long beach king,79.43,41699,1,9,3,1,7,Weekend,NaN
1,174884,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,44.50,41705,1,10,3,7,6,Weekday,NaN
2,174884,Co-Brand,TRAVEL AGENCIES,agoda,1427.46,41706,1,10,3,8,7,Weekend,NaN
3,174884,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,25.00,41707,1,11,3,9,1,Weekend,NaN
4,174884,Co-Brand,UTILITIES,hello amk hub,308.86,41707,1,11,3,9,1,Weekend,NaN


In [0]:
txn_demo_high.head()

,PartyID,PurchaseFrequency,CardType_nunique,Merchant_Category_nunique,Merchant_Name_nunique,Transaction_amount_mean,Transaction_amount_sum,Transaction_Date_max,DaysSinceLastPurchase,DaysSinceLastPurchase_Rev,R_quantile,F_quantile,M_quantile,RFM_q,EngagementScore,EngagementSegments,AgeBand,CreditLimitBand,District,Gender,RevolverFlag,CardType,Merchant_Category,Merchant_Name_Cleaned,Transaction_amount,Transaction_Date,HasTransacted?,WeekOfYear,Month,DayOfMonth,DayOfWeek,Weekday/Weekend,Merchant_Gateway
0,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,RESTAURANTS-BARS,long beach king,79.43,41699,1,9,3,1,7,Weekend,NaN
1,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,44.50,41705,1,10,3,7,6,Weekday,NaN
2,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,TRAVEL AGENCIES,agoda,1427.46,41706,1,10,3,8,7,Weekend,NaN
3,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,25.00,41707,1,11,3,9,1,Weekend,NaN
4,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,UTILITIES,hello amk hub,308.86,41707,1,11,3,9,1,Weekend,NaN


In [0]:
transactions['Merchant_Name_Cleaned'].nunique()

2302

In [0]:
txn_demo_high['Merchant_Name_Cleaned'].nunique()

2151

In [0]:
print(txn_demo_high['Merchant_Name_Cleaned'].value_counts(normalize = True))

fairprice                                 0.337299
Others                                    0.110124
esso                                      0.029186
caltex                                    0.016356
robinsons                                 0.015474
                                            ...   
marinedepot                               0.000004
az archery enterprises prescott vly us    0.000004
tara apothecary                           0.000004
your music guitares roma it               0.000004
expertecomm                               0.000004
Name: Merchant_Name_Cleaned, Length: 2151, dtype: float64


#Collaborative filtering

## Function to Calculate precision and Top n recommendations

In [0]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [0]:
from collections import defaultdict
# PartyID, Merchant_Name, true rating, estimated rating. Sorts the matrix by descending order of estimated ratings. Cuts off the top k (10) rows. Within those 10 rows, 
# it calculates the count of merchants truly liked aka relevant items (true rating >= 4), count of merchants predicted as liked aka recommended items (est >= 4), 
# and count of merchants both liked and recommended (est & true rating >=4)

def precision_recall_at_k(predictions, k=10, threshold=4):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

## Preparing the data in suitable format

In [0]:
df = pd.DataFrame({'Frequency' : txn_demo_high.groupby(['PartyID','Merchant_Name_Cleaned']).size()}).reset_index()
df

,PartyID,Merchant_Name_Cleaned,Frequency
0,174884,Others,4
1,174884,ackerman smith,1
2,174884,agoda,1
3,174884,apple store,1
4,174884,caltex,12
...,...,...,...
78592,85867033,starbucks,1
78593,85867033,sushi express,1
78594,85867033,toys r us,1
78595,85867033,uniqlo,1


In [0]:
TotalFreqbyPartyID = df.groupby(['PartyID']).agg({'Frequency':'sum'}).reset_index(level=['PartyID'])
TotalFreqbyPartyID

,PartyID,Frequency
0,174884,56
1,175387,23
2,175550,28
3,176146,34
4,176222,15
...,...,...
6409,85795366,43
6410,85795614,24
6411,85795846,11
6412,85808087,23


In [0]:
df = pd.merge(df,TotalFreqbyPartyID,how = 'left', on = 'PartyID')

In [0]:
df.rename(columns = {'Frequency_x': 'Merchant_Freq','Frequency_y': 'TotalFrequency'}, inplace= True)

In [0]:
df['FreqProp'] = (df['Merchant_Freq']/df['TotalFrequency'])

In [0]:
df['rating'] = pd.qcut(df['FreqProp'],5,['1','2','3','4','5'], duplicates='drop')

In [0]:
df.groupby('rating').agg({'FreqProp':['min','max']})

FreqProp          
             min       max
rating                    
1       0.001692  0.019608
2       0.019737  0.029412
3       0.029484  0.043478
4       0.043750  0.083333
5       0.083969  1.000000

In [0]:
df.head()

,PartyID,Merchant_Name_Cleaned,Merchant_Freq,TotalFrequency,rating,FreqProp
0,174884,Others,4,56,4,0.071429
1,174884,ackerman smith,1,56,1,0.017857
2,174884,agoda,1,56,1,0.017857
3,174884,apple store,1,56,1,0.017857
4,174884,caltex,12,56,5,0.214286


In [0]:
df['rating'].value_counts(normalize = True)

2    0.202718
1    0.201356
5    0.199168
4    0.198532
3    0.198226
Name: rating, dtype: float64

In [0]:
#df.to_csv('Data_for_Collaborative.csv',index = False)

## Creating data object for collaborative filtering

In [0]:
df = pd.read_csv('Data_for_Collaborative.csv')

In [0]:
#change the df_new here to df to skip the filtering earlier
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['PartyID', 'Merchant_Name_Cleaned', 'rating']], reader)

In [0]:
#building unbiased test set for testing precision and accuracy
import random

# Load the full dataset.
raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 70% of the data, B = 30% of the data
threshold = int(0.7 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# building 70% data as trainset
trainset = data.build_full_trainset()

# building 30% data as testset
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B

# Parameter tuning for collaborative algorithms by doing cv on trainset
  1. best item-item model- RMSE: 1.1169649211021628; Parameters:
{'k': 40, 'sim_options': {'name': 'cosine', 'user_based': False}}

  2. best user-user model- RMSE: 1.0579519004974747; Parameters: {'k': 2500, 'sim_options': {'name': 'cosine', 'user_based': True}}

  3. best LF model- RMSE: 1.0192473889324232; Parameters: {'n_factors': 90, 'lr_all': 0.1, 'reg_all': 0.02}

## User-User Similarity

In [0]:
param_grid = {'k': [40, 50, 85, 100, 150, 400, 1000, 1500, 2000, 2500, 3000],
             'sim_options': {'name': ['pearson_baseline','cosine'], 'user_based': [True]}
             }

uu_gs = RandomizedSearchCV(KNNWithMeans, param_grid, measures=['rmse'], n_iter = 15, cv=3, refit = False, n_jobs = -1, joblib_verbose= 50)

uu_gs.fit(data)

# best RMSE score
print(uu_gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(uu_gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   58.8s


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  3.9min
[Paralle

In [0]:
#best user-user model
# 1.0579519004974747
# {'k': 2500, 'sim_options': {'name': 'cosine', 'user_based': True}}

algo_u_u = uu_gs.best_estimator['rmse']

In [0]:
#refit on train data

algo_u_u.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
#pickle save
pickle.dump(algo_u_u,open('user_user_GSbest.sav','wb'))

In [0]:
uu_gs.cv_results.keys()

dict_keys(['split0_test_rmse', 'split1_test_rmse', 'split2_test_rmse', 'mean_test_rmse', 'std_test_rmse', 'rank_test_rmse', 'mean_fit_time', 'std_fit_time', 'mean_test_time', 'std_test_time', 'params', 'param_k', 'param_sim_options'])

In [0]:
uu_gs.cv_results['params'], uu_gs.cv_results['mean_test_rmse']

(array([{'k': 150, 'sim_options': {'name': 'cosine', 'user_based': True}},
        {'k': 85, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 100, 'sim_options': {'name': 'cosine', 'user_based': True}},
        {'k': 400, 'sim_options': {'name': 'cosine', 'user_based': True}},
        {'k': 1500, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 100, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 3000, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 85, 'sim_options': {'name': 'cosine', 'user_based': True}},
        {'k': 2000, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 50, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 40, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
        {'k': 2500, 'sim_options': {'name': 'cosine', 'user_based': True}},
        {'k': 1500, 'sim_optio

## Item-Item Similarity

In [0]:
param_grid = {'k': [40,50,60,80,100,200,500,1000,1500,2000,2500,3000],
             'sim_options': {'name': ['pearson_baseline','cosine'], 'user_based': [False]}
             }

i_i_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=3, joblib_verbose=40, n_jobs=-1)

i_i_gs.fit(data)

# best RMSE score
print(i_i_gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(i_i_gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

1.1169649211021628
{'k': 40, 'sim_options': {'name': 'cosine', 'user_based': False}}


[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.4min finished


In [0]:
#best item-item model
#1.1169649211021628
#{'k': 40, 'sim_options': {'name': 'cosine', 'user_based': False}}
algo_i_i = i_i_gs.best_estimator['rmse']

In [0]:
#refit on train data

algo_i_i.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
#pickle save
pickle.dump(algo_i_i,open('item_item_GSbest.sav','wb'))

In [0]:
i_i_gs.cv_results['params'], i_i_gs.cv_results['mean_test_rmse']

([{'k': 40, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 40, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 50, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 50, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 60, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 60, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 80, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 80, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 100, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 100, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 200, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 200, 'sim_options': {'name': 'cosine', 'user_based': False}},
  {'k': 500, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}},
  {'k': 500, 'sim_options': 

## Latent Factorisation

In [0]:
param_grid = {'n_factors': [10,20,30,40,50,60,70,80,90,100],
              'lr_all': [0.00001, 0.0001, 0.001, 0.01,0.1,0.2,0.5],
              'reg_all': [0.01,0.02,0.05,0.1,0.5,1,2]
             }

mf_gs = RandomizedSearchCV(SVD, param_grid,n_iter=50, measures=['rmse'], cv=3, n_jobs=-1, joblib_verbose=50)

mf_gs.fit(data)

# best RMSE score
print(mf_gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(mf_gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

In [0]:
#best latent factor model
#0.8303826655531948
#1.0192473889324232
#{'n_factors': 90, 'lr_all': 0.1, 'reg_all': 0.02}
algo_LF = mf_gs.best_estimator['rmse']

In [0]:
#refit on train data

algo_LF.fit(trainset)

In [0]:
#pickle save
import pickle
pickle.dump(algo_LF,open('LF_GSbest.sav','wb'))

#Load Best Models

In [0]:
#pickle load USER-USER
algo_u_u = pickle.load(open('user_user_GSbest.sav','rb'))

In [0]:
#pickle load ITEM-ITEM
algo_i_i = pickle.load(open('item_item_GSbest.sav','rb'))

In [0]:
#pickle load Latent Factorisation
algo_LF = pickle.load(open('LF_GSbest.sav','rb'))

# Calculating Precision on test set 
  LF: Precision = 95%, recall = 47.5% (but this doesn't matter, because only items with top 10 estimated scores are recommended. Since number of predicted items are less, TP rate is also less)
  
  User-user: Precision = 92.5%
  Item-item: Precision = 91.8%

In [0]:
predictions_ii_test = algo_i_i.test(testset)
precisions, recalls = precision_recall_at_k(predictions_ii_test, k=10, threshold=4)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.9574077882901415
0.6344131358837239


In [0]:
predictions_uu_test = algo_u_u.test(testset)
precisions, recalls = precision_recall_at_k(predictions_uu_test, k=10, threshold=4)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.9601672836966956
0.6449026463732346


In [0]:
predictions_lf_test = algo_LF.test(testset)
precisions, recalls = precision_recall_at_k(predictions_lf_test, k=10, threshold=4)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.9889277389277388
0.7088783765254352


In [0]:
rmse(predictions_ii_test)

RMSE: 0.9130


0.9130386842158857

In [0]:
rmse(predictions_uu_test)

RMSE: 0.9048


0.9048204844530616

In [0]:
rmse(predictions_lf_test)

RMSE: 0.5853


0.5853321903956664

# User-wise recommendations - Demo

In [0]:
df['PartyID'].unique()

array([  174884,   175387,   175550, ..., 85795846, 85808087, 85867033])

In [0]:
i=1
user_input = input("Please enter user_id to recommend for her/him: ")
while (i == 1):
  if (pd.to_numeric(user_input) not in list(df['PartyID'])):
    print("User_id is not in training set. Enter some other value.")
    user_input = input("Please enter user_id to recommend for her/him: ")
  else:
    i=0

#entered 175550
#can also enter 174884,85795846

Please enter user_id to recommend for her/him: 218737


In [0]:
#get a list of all merchant names
iids = df['Merchant_Name_Cleaned'].unique()
#get a list of merchants that uid 175550 has transacted with
iids_175550 = df.loc[df['PartyID']==int(user_input),'Merchant_Name_Cleaned']
#remove the merchants that uid 175550 has transacted with from list of merchant names
iids_to_pred = np.setdiff1d(iids,iids_175550) 

In [0]:
len(iids_to_pred), len(iids_175550),len(iids) 

(2133, 18, 2151)

In [0]:
#Latent Factor - get predicted ratings for untransacted merchants
predictions_lf = [algo_LF.predict(pd.to_numeric(user_input),iid) for iid in iids_to_pred]
predictions_lf[0]
#convert to array to see all

Prediction(uid=218737, iid='7-eleven', r_ui=None, est=2.118406167598147, details={'was_impossible': False})

In [0]:
df[df['PartyID']==int(user_input)]

,PartyID,Merchant_Name_Cleaned,Merchant_Freq,TotalFrequency,rating,FreqProp
2901,218737,Others,4,40,5,0.100
2902,218737,bee s birds nest singapore,1,40,2,0.025
2903,218737,cathay cineplex,1,40,2,0.025
2904,218737,enets,1,40,2,0.025
2905,218737,fairprice,9,40,5,0.225
2906,218737,giosis qoo,7,40,5,0.175
2907,218737,groupon,4,40,5,0.100
2908,218737,ichiban sushi,1,40,2,0.025
2909,218737,ktph,1,40,2,0.025
2910,218737,mc donalds,1,40,2,0.025


In [0]:
df.loc[df['PartyID']==int(user_input),'Merchant_Name_Cleaned'].to_list()

['Others',
 'bee s birds nest singapore',
 'cathay cineplex',
 'enets',
 'fairprice',
 'giosis qoo',
 'groupon',
 'ichiban sushi',
 'ktph',
 'mc donalds',
 'new york new york',
 'oac insurance',
 'pandora',
 'pastamania',
 'pizza hut s',
 'ramen play',
 'swensen s',
 'zalora']

In [0]:
#Latent Factor - get predicted rating for specific merchant
algo_LF.predict(175550,'groupon',5)

Prediction(uid=175550, iid='groupon', r_ui=5, est=5, details={'was_impossible': False})

In [0]:
#Latent factor - find top merchant to recommend
pred_ratings_lf = np.array([pred.est for pred in predictions_lf])
#find the index of the maximum predicted rating
i_max = pred_ratings_lf.argmax()
#use this to find the corresponding iid to recommend
iid = iids_to_pred[i_max]
print('Top merchant for user {2} is "{0}" with predicted rating of {1}.'.format (iid,pred_ratings_lf[i_max],user_input))

Top merchant for user 218737 is "fitness first" with predicted rating of 4.577881059406431.


In [0]:
#Latent factor - find top 5 merchant to recommend
#for top 5 predicted ratings
pred_ratings_lf = np.array([pred.est for pred in predictions_lf])
#find the index of the maximum predicted rating
i_maxs = np.argsort(-1*pred_ratings_lf)[:5]
iids_lf = iids_to_pred[i_maxs]
print('Top 5 merchants for user {2} are: {0} with predicted rating of {1} respectively.'.format (iids_lf,pred_ratings_lf[i_maxs],user_input))

Top 5 merchants for user 218737 are: ['fitness first' 'spore telecom' 'hospital illness careplus'
 'the butter factory p l' 'taxi service'] with predicted rating of [4.57788106 4.54957991 4.3965501  4.31093863 4.24410426] respectively.


In [0]:
#ITEM-ITEM - to get predicted ratings
predictions_ii = [algo_i_i.predict(pd.to_numeric(user_input),iid) for iid in iids_to_pred]
predictions_ii[0]
#convert to array to see all

Prediction(uid=175550, iid='7-eleven', r_ui=None, est=2.5022913316773012, details={'actual_k': 2, 'was_impossible': False})

In [0]:
#ITEM-ITEM
#for top 5 predicted ratings
pred_ratings_ii = np.array([pred.est for pred in predictions_ii])
#find the index of the maximum predicted rating
i_maxs = np.argsort(-1*pred_ratings_ii)[:5]
iids_ii = iids_to_pred[i_maxs]
print('Top 5 merchants for user {2} are: {0} with predicted rating of {1} respectively.'.format (iids_ii,pred_ratings_ii[i_maxs],user_input))

Top 5 merchants for user 175550 are: ['jess pub' 'nuskin' 'nucerity in' 'ngovietanhf' 'think pte'] with predicted rating of [5. 5. 5. 5. 5.] respectively.


In [0]:
#USER-USER - to get predicted ratings
predictions_uu = [algo_u_u.predict(pd.to_numeric(user_input),iid) for iid in iids_to_pred]
predictions_uu[0]
#convert to array to see all

Prediction(uid=175550, iid='7-eleven', r_ui=None, est=4.135680182969481, details={'actual_k': 11, 'was_impossible': False})

In [0]:
#ITEM-ITEM
#for top 5 predicted ratings
pred_ratings_uu = np.array([pred.est for pred in predictions_uu])
#find the index of the maximum predicted rating
i_maxs = np.argsort(-1*pred_ratings_ii)[:5]
iids_uu = iids_to_pred[i_maxs]
print('Top 5 merchants for user {2} are: {0} with predicted rating of {1} respectively.'.format (iids_uu,pred_ratings_uu[i_maxs],user_input))

Top 5 merchants for user 175550 are: ['jess pub' 'nuskin' 'nucerity in' 'ngovietanhf' 'think pte'] with predicted rating of [5. 5. 5. 5. 5.] respectively.


In [0]:
df['rating'].astype(int).mean()

2.991437332213698

#Generating data for clustering

In [0]:
txn_demo_high

,PartyID,PurchaseFrequency,CardType_nunique,Merchant_Category_nunique,Merchant_Name_nunique,Transaction_amount_mean,Transaction_amount_sum,Transaction_Date_max,DaysSinceLastPurchase,DaysSinceLastPurchase_Rev,R_quantile,F_quantile,M_quantile,RFM_q,EngagementScore,EngagementSegments,AgeBand,CreditLimitBand,District,Gender,RevolverFlag,CardType,Merchant_Category,Merchant_Name_Cleaned,Transaction_amount,Transaction_Date,HasTransacted?,WeekOfYear,Month,DayOfMonth,DayOfWeek,Weekday/Weekend,Merchant_Gateway
0,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,RESTAURANTS-BARS,long beach king,79.43,41699,1,9,3,1,7,Weekend,NaN
1,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,44.50,41705,1,10,3,7,6,Weekday,NaN
2,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,TRAVEL AGENCIES,agoda,1427.46,41706,1,10,3,8,7,Weekend,NaN
3,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,FOOD STORES-WAREHOUSE,fairprice,25.00,41707,1,11,3,9,1,Weekend,NaN
4,174884,56,1,12,27,152.070893,8515.97,0,0,0,5,5,5,555,15,High,05 (60 - 69),01 (< 10)K,57,M,02 Transactor,Co-Brand,UTILITIES,hello amk hub,308.86,41707,1,11,3,9,1,Weekend,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242848,85867033,42,1,11,36,176.071191,7394.99,0,3,-3,3,5,5,355,13,High,01 (20 - 29),01 (< 10)K,51,F,01 New Customer,Core Card,RESTAURANTS-BARS,Others,39.22,41775,1,20,5,16,6,Weekday,NaN
242849,85867033,42,1,11,36,176.071191,7394.99,0,3,-3,3,5,5,355,13,High,01 (20 - 29),01 (< 10)K,51,F,01 New Customer,Core Card,RESTAURANTS-BARS,Others,131.26,41777,1,21,5,18,1,Weekend,NaN
242850,85867033,42,1,11,36,176.071191,7394.99,0,3,-3,3,5,5,355,13,High,01 (20 - 29),01 (< 10)K,51,F,01 New Customer,Core Card,FOOD STORES-WAREHOUSE,fairprice,6.78,41778,1,21,5,19,2,Weekday,NaN
242851,85867033,42,1,11,36,176.071191,7394.99,0,3,-3,3,5,5,355,13,High,01 (20 - 29),01 (< 10)K,51,F,01 New Customer,Core Card,FOOD STORES-WAREHOUSE,Others,22.55,41779,1,21,5,20,3,Weekday,NaN


In [0]:
df_clustering = pd.pivot_table(txn_demo_high,values = 'Transaction_amount',index = 'PartyID',columns='Merchant_Category',aggfunc='sum')
df_clustering = df_clustering.fillna(0)

In [0]:
df_clustering['Total'] = df_clustering.sum(axis=1)

In [0]:
df_clustering

Merchant_Category,AIRLINE,AUTO RENTAL,CLOTHING STORES,DEPARTMENT STORES,DISCOUNT STORES,DRUG STORES,EDUCATION,ELECTRIC-APPLIANCE,FOOD STORES-WAREHOUSE,GAS STATIONS,HARDWARE,HEALTH CARE,HOTEL-MOTEL,INTERIOR FURNISHINGS,MAIL ORDER,OTHER RETAIL,OTHER SERVICES,OTHER TRANSPORT,Others,PROFESSIONAL SERVICES,QUASI CASH,RECREATION,REPAIR SHOPS,RESTAURANTS-BARS,SPORTING-TOY STORES,TRAVEL AGENCIES,UTILITIES,VEHICLES,Total
PartyID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
174884,0.00,0.00,121.43,0.00,0.0,0.00,0.0,0.00,958.88,673.23,1055.45,0.00,0.0,399.96,216.13,27.78,0.00,860.36,0.0,0.00,0.00,0.00,0.0,1627.50,0.00,1427.46,308.86,838.93,8515.97
175387,0.00,0.00,146.87,0.00,0.0,96.42,0.0,0.00,586.60,0.00,0.00,0.00,0.0,0.00,0.00,47.60,545.94,0.00,0.0,0.00,0.00,119.21,0.0,151.08,0.00,0.00,0.00,0.00,1693.72
175550,0.00,0.00,0.00,0.00,0.0,0.00,0.0,55.53,0.00,1934.40,0.00,0.00,0.0,0.00,0.00,32.27,195.98,0.00,0.0,1.34,0.00,0.00,0.0,0.00,38.43,0.00,0.00,0.00,2257.95
176146,0.00,0.00,540.68,213.08,0.0,0.00,0.0,604.39,470.94,0.00,0.00,315.14,0.0,0.00,0.00,73.95,0.00,0.00,0.0,0.00,0.00,19.78,0.0,144.82,140.83,0.00,76.66,0.00,2600.27
176222,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,148.52,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,9120.13,0.00,0.00,0.0,145.53,0.00,0.00,0.00,0.00,9414.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85795366,0.00,0.00,8.85,0.00,0.0,0.00,0.0,0.00,165.43,0.00,0.00,0.00,0.0,0.00,0.00,0.00,130.87,0.00,0.0,0.00,0.00,0.00,0.0,161.15,0.00,0.00,172.20,0.00,638.50
85795614,0.00,0.00,236.59,0.00,0.0,0.00,0.0,100.02,496.32,0.00,0.00,0.00,0.0,0.00,0.00,137.29,454.65,0.00,0.0,232.27,0.00,395.51,0.0,182.10,0.00,0.00,159.37,0.00,2394.12
85795846,567.72,0.00,478.39,237.42,0.0,0.00,0.0,0.00,88.43,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,26.94,0.00,0.00,0.00,0.00,1398.90


In [0]:
#df_clustering.reset_index(inplace = True)

In [0]:
df_clustering.columns

Index(['AIRLINE', 'AUTO RENTAL', 'CLOTHING STORES', 'DEPARTMENT STORES',
       'DISCOUNT STORES', 'DRUG STORES', 'EDUCATION', 'ELECTRIC-APPLIANCE',
       'FOOD STORES-WAREHOUSE', 'GAS STATIONS', 'HARDWARE', 'HEALTH CARE',
       'HOTEL-MOTEL', 'INTERIOR FURNISHINGS', 'MAIL ORDER', 'OTHER RETAIL',
       'OTHER SERVICES', 'OTHER TRANSPORT', 'Others', 'PROFESSIONAL SERVICES',
       'QUASI CASH', 'RECREATION', 'REPAIR SHOPS', 'RESTAURANTS-BARS',
       'SPORTING-TOY STORES', 'TRAVEL AGENCIES', 'UTILITIES', 'VEHICLES',
       'Total'],
      dtype='object', name='Merchant_Category')

In [0]:
for col in df_clustering.columns:
       df_clustering[col] = df_clustering[col]/df_clustering['Total']

In [0]:
df_clustering.drop(columns = 'Total',inplace= True)

In [0]:
df_clustering

Merchant_Category,AIRLINE,AUTO RENTAL,CLOTHING STORES,DEPARTMENT STORES,DISCOUNT STORES,DRUG STORES,EDUCATION,ELECTRIC-APPLIANCE,FOOD STORES-WAREHOUSE,GAS STATIONS,HARDWARE,HEALTH CARE,HOTEL-MOTEL,INTERIOR FURNISHINGS,MAIL ORDER,OTHER RETAIL,OTHER SERVICES,OTHER TRANSPORT,Others,PROFESSIONAL SERVICES,QUASI CASH,RECREATION,REPAIR SHOPS,RESTAURANTS-BARS,SPORTING-TOY STORES,TRAVEL AGENCIES,UTILITIES,VEHICLES
PartyID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
174884,0.000000,0.000000,0.014259,0.000000,0.0,0.000000,0.0,0.000000,0.112598,0.079055,0.123938,0.000000,0.0,0.046966,0.025379,0.003262,0.000000,0.101029,0.0,0.000000,0.000000,0.000000,0.0,0.191112,0.000000,0.167622,0.036268,0.098513
175387,0.000000,0.000000,0.086714,0.000000,0.0,0.056928,0.0,0.000000,0.346338,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.028104,0.322332,0.000000,0.0,0.000000,0.000000,0.070384,0.0,0.089200,0.000000,0.000000,0.000000,0.000000
175550,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.024593,0.000000,0.856706,0.000000,0.000000,0.0,0.000000,0.000000,0.014292,0.086796,0.000000,0.0,0.000593,0.000000,0.000000,0.0,0.000000,0.017020,0.000000,0.000000,0.000000
176146,0.000000,0.000000,0.207932,0.081945,0.0,0.000000,0.0,0.232434,0.181112,0.000000,0.000000,0.121195,0.0,0.000000,0.000000,0.028439,0.000000,0.000000,0.0,0.000000,0.000000,0.007607,0.0,0.055694,0.054160,0.000000,0.029482,0.000000
176222,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.015776,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.968765,0.000000,0.000000,0.0,0.015459,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85795366,0.000000,0.000000,0.013861,0.000000,0.0,0.000000,0.0,0.000000,0.259092,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.204965,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.252388,0.000000,0.000000,0.269695,0.000000
85795614,0.000000,0.000000,0.098821,0.000000,0.0,0.000000,0.0,0.041777,0.207308,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.057345,0.189903,0.000000,0.0,0.097017,0.000000,0.165201,0.0,0.076061,0.000000,0.000000,0.066567,0.000000
85795846,0.405833,0.000000,0.341976,0.169719,0.0,0.000000,0.0,0.000000,0.063214,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.019258,0.000000,0.000000,0.000000,0.000000


In [0]:
#df_clustering.to_csv('Clustering_data.csv')